# Bell State Preparation experiment: Compute $\langle \bar{\Phi}^+ | \bar{Z}\bar{Z} | \bar{\Phi}^+\rangle$ or $\langle \bar{\Phi}^+ | \bar{X}\bar{X} | \bar{\Phi}^+\rangle$ with the seven-qubit Steane code

In [1]:
from typing import List, Dict, Sequence
import itertools
import functools
import numpy as np
from tqdm import tqdm
import cirq
import qiskit
from qiskit.circuit.library import Barrier
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

from mitiq import PauliString

from encoded.diagonalize import get_stabilizer_matrix_from_paulis, get_measurement_circuit, get_paulis_from_stabilizer_matrix

In [2]:
import datetime


time_key = datetime.datetime.now().strftime("%m_%d_%Y_%H:%M:%S")  # For saving results.

## Set parameters

In [3]:
n = 7                                   # Number of physical qubits
nshots = 10_000                         # Number of samples/shots
depth = 0                               # Number of folded Bell state preparation circuits for added noise
k = 2                                   # Number of logical qubits.
obs = "X"                               # Observable to use: "X" or "Z". (This translates to logical XX or logical ZZ, respectively.)

In [4]:
# Computer and qubits to use.
service = qiskit_ibm_runtime.QiskitRuntimeService()  # This assumes a saved account.
computer = service.backend("ibm_fez")
sampler = Sampler(computer)

# See calibration data at https://quantum.ibm.com/services/resources to select good qubits.
# layout = {
#     1 : [2, 3],
#     7 : [2, 3, 4, 5, 6, 7, 16, 17, 22, 23, 24, 25, 26, 27],
# }


# The best qubits ever on Fez Feb 6-8.
layout = {
    1 : [123, 124],
    7 : [123, 124, 125, 126, 127, 128, 136, 137, 142, 143, 144, 145, 146, 147],
}


# Good on Kyiv 2/6
# layout = {
#     1 : [20, 21],
#     7 : [0, 1, 2, 3, 4, 5, 14, 15, 18, 19, 20, 21, 22, 23],
# }

# Good on Sherbrooke 2/5.
# layout = {
#     1 : [103, 104],
#     7 : [103, 104, 105, 106, 107, 108, 111, 112, 121, 122, 123, 124, 125, 126],
# }

# Good on Kyiv 2/5.
# layout = {
#     1 : [95, 96],
#     7 : [95, 96, 97, 98, 99, 100, 101, 113, 114, 115, 116, 117, 118, 119],
# }

## Helper functions

In [5]:
# Expectation of pauli on bitstring measured in diagonal basis.
def compute_expectation(
    pauli: cirq.PauliString,
    counts: Dict[str, int],
) -> float:
    if pauli is cirq.PauliString():
        return 1.0

    expectation = 0.0

    indices = [q.x for q in pauli.qubits]
    for key, value in counts.items():
        key = list(map(int, list(key[::-1])))
        expectation += (-1) ** sum([key[i] for i in indices]) * value

    return expectation / sum(counts.values())

# Prepares logical |0> state on Steane Code
def encode_steane(qreg: Sequence[cirq.Qid]) -> cirq.Circuit:
    circuit = cirq.Circuit()

    circuit.append(cirq.H.on(qreg[0]))
    circuit.append(cirq.H.on(qreg[4]))
    circuit.append(cirq.H.on(qreg[6]))

    circuit.append(cirq.CNOT.on(qreg[0], qreg[1]))
    circuit.append(cirq.CNOT.on(qreg[4], qreg[5]))

    circuit.append(cirq.CNOT.on(qreg[6], qreg[3]))
    circuit.append(cirq.CNOT.on(qreg[6], qreg[5]))
    circuit.append(cirq.CNOT.on(qreg[4], qreg[2]))
    
    circuit.append(cirq.CNOT.on(qreg[0], qreg[3]))
    circuit.append(cirq.CNOT.on(qreg[4], qreg[1]))
    circuit.append(cirq.CNOT.on(qreg[3], qreg[2]))

    return circuit

def strs_to_paulis(pauli_strs : List[str]) -> List[cirq.PauliString]:
    stab_list = []
    for stab_str in pauli_strs:
        stab_list.append(PauliString(stab_str)._pauli)
    return stab_list

def generate_stabilizer_elements(generators: List[cirq.PauliString]) -> List[cirq.PauliString]:
    elements = []
    for string in itertools.chain.from_iterable(itertools.combinations(generators, r) for r in range(len(generators) + 1)):
        elements.append(
            functools.reduce(lambda a, b: a * b, string, cirq.PauliString())
        )
    return elements

# For qiskit circuits
def get_active_qubits(circ):
    dag = qiskit.converters.circuit_to_dag(circ)
    active_qubits = [qubit for qubit in circ.qubits if qubit not in dag.idle_wires()]
    return active_qubits

def get_lst_ev(counts, observables, stabilizers):
    numerator = 0
    for obs in observables:
        numerator += compute_expectation(obs, counts) / len(observables)
    denominator = 0
    for stab in stabilizers:
        denominator += compute_expectation(stab, counts) / len(stabilizers)
    return float(np.real_if_close(numerator / denominator))


### Run unmitigated experiment

In [6]:
qreg = cirq.LineQubit.range(k)

circuit = cirq.Circuit()
circuit.append(cirq.H.on(qreg[0]))
for i in range(len(qreg)-1):
    circuit.append(cirq.CNOT.on(qreg[i], qreg[i+1]))

if obs == "X":
    circuit.append(cirq.H.on_each(qreg))

circuit = qiskit.QuantumCircuit.from_qasm_str(circuit.to_qasm())
circuit.measure_active()
# Compile to device.
compiled_physical = qiskit.transpile(
    circuit, 
    backend=computer,
    initial_layout=layout[1],  # Hardcode n = 1 (i.e., no encoding) to get layout.
    routing_method="sabre",
    # scheduling_method="asap",
    optimization_level=0,
)
print(compiled_physical.draw(fold=-1, idle_wires=False))

global phase: 5π/4
           ┌─────────┐┌────┐┌─────────┐   ┌─────────┐┌────┐┌─────────┐                             ░ ┌─┐   
q_0 -> 123 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─■─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─────────────────────────────░─┤M├───
           ├─────────┤├────┤├─────────┤ │ ├─────────┤├────┤├─────────┤┌─────────┐┌────┐┌─────────┐ ░ └╥┘┌─┐
q_1 -> 124 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─■─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░──╫─┤M├
           └─────────┘└────┘└─────────┘   └─────────┘└────┘└─────────┘└─────────┘└────┘└─────────┘ ░  ║ └╥┘
measure: 2/═══════════════════════════════════════════════════════════════════════════════════════════╩══╩═
                                                                                                      0  1 


In [7]:
job_physical = sampler.run(
    [compiled_physical],
    shots=nshots
)
# job_physical = service.job("cuu7ud8mc35c73b5g0c0")

In [14]:
all_counts_raw = [result.data.measure.get_counts() for result in job_physical.result()]
all_counts_raw

[{'00': 5024, '11': 4677, '01': 151, '10': 148}]

In [15]:
ev_physical = compute_expectation(PauliString(obs * k)._pauli, all_counts_raw[0])
print("Physical result:", ev_physical)

Physical result: 0.9402


## Run encoded experiment

In [8]:
generator_strs = [
    "XXXXIII",
    "IXXIXXI",
    "IIXXIXX",
    "ZZZZIII",
    "IZZIZZI",
    "IIZZIZZ"
]

observable = PauliString(obs * n * k)._pauli

qreg = cirq.LineQubit.range(n * k)

stabilizer_generators = strs_to_paulis(generator_strs)
stabilizer_matrix = get_stabilizer_matrix_from_paulis(stabilizer_generators, qreg[:n])

m_circuit, transformed_matrix = get_measurement_circuit(stabilizer_matrix)
if obs == "X":
    m_circuit.insert(0, cirq.H.on_each(qreg[:n]))

measurement_circuit = cirq.Circuit.concat_ragged(
    m_circuit,
    m_circuit.transform_qubits(dict(zip(qreg[:n], qreg[n:])))
)

transformed_generators = get_paulis_from_stabilizer_matrix(transformed_matrix)
# print(transformed_generators)
stabilizer_elements = generate_stabilizer_elements(
    transformed_generators + \
    [p.map_qubits(dict(zip(qreg[:n], qreg[n:]))) for p in transformed_generators]
)

transformed_observable = observable.conjugated_by(measurement_circuit**-1)
observable_elements = [transformed_observable * stab for stab in stabilizer_elements]

In [9]:
encoding = cirq.Circuit.concat_ragged(
    encode_steane(qreg[:n]),
    encode_steane(qreg[n:])
)

# prepare Bell state
encoding.append(cirq.H.on_each(qreg[:n]))
encoding.append(cirq.CNOT.on_each([(qreg[i], qreg[i+n]) for i in range(n)]))

encoding = qiskit.QuantumCircuit.from_qasm_str(encoding.to_qasm())
measurement = qiskit.QuantumCircuit.from_qasm_str(measurement_circuit.to_qasm())

circ_full = encoding.compose(
    Barrier(n*k, label="encoding"), get_active_qubits(encoding)
).compose(
    Barrier(n*k, label="measurement"), get_active_qubits(encoding)
).compose(measurement)
circ_full.measure_active()

compiled = qiskit.transpile(
    circ_full, 
    backend=computer,
    initial_layout=layout[n],
    routing_method="sabre",
    # scheduling_method="asap",
    optimization_level=3,
)
compiled.draw(fold=-1, idle_wires=False)

┌─────────┐┌────┐┌─────────┐    ┌────┐                                                                                            ┌────┐              ┌────┐     ┌─────────┐   ┌────┐   ┌───────┐                         ┌────┐  ┌──────────┐                                                                 ┌────┐  ┌─────────┐               ┌────┐            ┌────┐            ┌────┐           ┌────┐           ┌────┐           ┌────┐                                                                                          ┌────┐      ┌────┐      ┌─────────┐   ┌────┐   ┌───────┐             ┌────┐   ┌─────────┐                                                                                                                    ░           ░      ┌────────┐┌────┐┌─────────┐         ┌────┐         ┌────┐   ┌─────────┐   ┌────┐  ┌────────┐                                                                                                                                                                                      ┌────┐   ┌───────┐               ┌────┐                                                                                                                                                                                       ┌────┐  ┌─────────┐   ┌────┐                ┌────┐                           ┌────┐           ┌────┐        ┌────┐                    ┌────┐        ┌────┐                 ┌────┐                                                        ┌─────────┐   ┌────┐  ┌─────────┐ ░                            ┌─┐            
 q_0 -> 123 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■─┤ √X ├─────────────────────────────────────────────────────────────────────────────────────■──────┤ √X ├─────■────────┤ √X ├───■─┤ Rz(π/2) ├───┤ √X ├───┤ Rz(π) ├─────────■────────■──────┤ √X ├──┤ Rz(-π/2) ├─────────────────────────────────────────────────────────────■───┤ √X ├──┤ Rz(π/2) ├─────────────■─┤ √X ├────────■───┤ √X ├────────■───┤ √X ├─────■─────┤ √X ├─────■─────┤ √X ├─────■─────┤ √X ├────────────────────────────────────────────────────────────────────────────────────■─────┤ √X ├──■───┤ √X ├──■───┤ Rz(π/2) ├───┤ √X ├───┤ Rz(π) ├─────■───────┤ √X ├───┤ Rz(π/2) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░───────────░──────┤ Rz(-π) ├┤ √X ├┤ Rz(π/2) ├─────■───┤ √X ├───────■─┤ √X ├─■─┤ Rz(π/2) ├───┤ √X ├──┤ Rz(-π) ├───────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────■───┤ √X ├───┤ Rz(π) ├───────────■───┤ √X ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ √X ├──┤ Rz(π/2) ├─■─┤ √X ├────────────■───┤ √X ├──────────────────■────────┤ √X ├────■──────┤ √X ├────■───┤ √X ├──■───────────■─────┤ √X ├────■───┤ √X ├────────────■────┤ √X ├─────────────■──────────────────────────■─────────────■─┤ Rz(π/2) ├───┤ √X ├──┤ Rz(π/2) ├─░────────────────────────────┤M├────────────
            ├─────────┤├────┤├─────────┴┐ │ ├────┤┌─────────┐                    ┌────┐            ┌────┐                ┌────┐        │      ├────┤     │        ├────┤   │ └─────────┘   └────┘   └───────┘         │        │    ┌─┴────┴─┐└──┬────┬──┘┌─────────┐   ┌────┐   ┌────┐   ┌─────────┐┌────┐┌─────────┐ │   ├────┤  └─────────┘             │ └────┘        │   └────┘        │   └────┘     │     ├────┤     │     ├────┤     │     ├────┤             ┌────┐            ┌────┐                                               │     └────┘  │   └────┘  │   └─────────┘   └────┘   └───────┘     │       └────┘   └─────────┘                                                                                                                    ░           ░      ├────────┤├────┤├─────────┤     │   ├────┤       │ ├────┤ │ └─────────┘   ├────┤  └────────┘                                    

In [10]:
compiled.count_ops()

OrderedDict([('sx', 434),
             ('cz', 234),
             ('rz', 155),
             ('measure', 14),
             ('barrier', 3)])

In [11]:
job = sampler.run(
    [compiled],
    shots=nshots,
)
# job = service.job("cuu7ulvbet5c738gvlvg")

In [16]:
all_counts = [result.data.measure.get_counts() for result in job.result()]

In [17]:
ev = get_lst_ev(all_counts[0], tqdm(observable_elements), tqdm(stabilizer_elements))
print(ev)

100%|██████████| 4096/4096 [01:29<00:00, 45.70it/s]

0.992678462477116


## With DD

In [12]:
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"

In [13]:
job_dd = sampler.run(
    [compiled],
    shots=nshots,
)
# job_dd = service.job("cuu7uqecq8as73and1bg")

In [18]:
all_counts_dd = [result.data.measure.get_counts() for result in job_dd.result()]

In [19]:
ev_dd = get_lst_ev(all_counts_dd[0], tqdm(observable_elements), tqdm(stabilizer_elements))
print(ev_dd)

100%|██████████| 4096/4096 [01:15<00:00, 54.46it/s]

0.9937022042285197


## With REM + DD

In [14]:
from qiskit_experiments.library import LocalReadoutError

In [15]:
experiment = LocalReadoutError(layout[n])

In [16]:
result = experiment.run(computer)

In [75]:
mitigator = result.analysis_results("Local Readout Mitigator").value

In [76]:
# for i, qubit in enumerate(mitigator.qubits):
#     print(f"Qubit: {qubit}")
#     print(mitigator.mitigation_matrix(qubits=i))

In [77]:
counts = all_counts[0]
mitigated_quasi_probs = mitigator.quasi_probabilities(counts)
mitigated_stddev = mitigated_quasi_probs._stddev_upper_bound
mitigated_probs = (mitigated_quasi_probs.nearest_probability_distribution().binary_probabilities())

In [78]:
mitigated_counts = {k: round(v * nshots) for k, v in mitigated_probs.items()}

In [ ]:
sum(mitigated_counts.values())

In [ ]:
ev_rem = get_lst_ev(mitigated_counts, tqdm(observable_elements), tqdm(stabilizer_elements))
print(ev_rem)

In [81]:
counts_dd = all_counts_dd[0]
mitigated_quasi_probs = mitigator.quasi_probabilities(counts_dd)
mitigated_stddev = mitigated_quasi_probs._stddev_upper_bound
mitigated_probs = (mitigated_quasi_probs.nearest_probability_distribution().binary_probabilities())
mitigated_counts_dd = {k: round(v * nshots) for k, v in mitigated_probs.items()}

In [ ]:
sum(mitigated_counts_dd.values())

In [ ]:
ev_rem_dd = get_lst_ev(mitigated_counts_dd, tqdm(observable_elements), tqdm(stabilizer_elements))
print(ev_rem_dd)

## Save results

In [20]:
save_key = f"bell_steane_code_{obs.lower() * k}_{computer.name}_{time_key}_job_raw_id_{job_physical.job_id()}_job_encoded_id_{job.job_id()}_job_encoded_dd_id_{job_dd.job_id()}"  # _rem_calibration_id_{result.job_ids[0]}"

In [21]:
import os

In [22]:
os.mkdir(save_key)

In [23]:
np.savetxt(f"{save_key}/qubits_physical.txt", layout[1])
np.savetxt(f"{save_key}/qubits_encoded.txt", layout[n])

In [24]:
np.savetxt(f"{save_key}/nshots.txt", [nshots])

In [25]:
np.savetxt(f"{save_key}/physical.txt", [ev_physical])
np.savetxt(f"{save_key}/encoded.txt", [ev])
np.savetxt(f"{save_key}/encoded_dd.txt", [ev_dd])
# np.savetxt(f"{save_key}/encoded_rem.txt", [ev_rem])
# np.savetxt(f"{save_key}/encoded_dd_rem.txt", [ev_rem_dd])